# 🔍 RAG Pipeline For LLMs 🚀


## 📖 Project Overview

This Notebook implements a **Retrieval-Augmented Generation (RAG)** pipeline using state-of-the-art language models and vector databases. The system intelligently retrieves relevant information from Wikipedia articles and generates accurate, context-aware answers to user questions.



### ✨ Key Features

- 📚 **Dynamic Knowledge Retrieval** from Wikipedia
- 🧮 **Semantic Search** using sentence transformers
- ⚡ **Fast Vector Similarity** with FAISS indexing
- 🤖 **Intelligent Answer Generation** using pre-trained QA models
- 📊 **Confidence Scoring** for answer quality assessment
- 🎛️ **Customizable Parameters** (chunk size, retrieval count, etc.)

### 🏗️ Architecture

```
User Query → Embedding → FAISS Search → Retrieve Chunks → QA Model → Answer + Confidence
```

### 🛠️ Tech Stack & Models

**🔧 Core Technologies:**
- **🤗 Transformers**: For tokenization and question-answering models
- **📝 Sentence Transformers**: For semantic embeddings
- **⚡ FAISS**: For efficient vector similarity search
- **📖 Wikipedia API**: For knowledge base content
- **🐍 Python**: Core implementation language

**🤖 AI Models Used:**
- **📏 Text Chunking**: `sentence-transformers/all-mpnet-base-v2` tokenizer
- **🧮 Vector Embeddings**: `sentence-transformers/all-mpnet-base-v2` (768-dimensional)
- **❓ Question Answering**: `deepset/roberta-base-squad2` (RoBERTa fine-tuned on SQuAD 2.0)
- **🔍 Vector Search**: FAISS IndexFlatL2 for L2 distance similarity

## 📦 Installation & Setup

### 🔧 Installing Required Dependencies

First, let's install all the necessary Python packages for our RAG pipeline:

In [2]:
!pip install wikipedia
!pip install transformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install tf_keras
!pip install gradio

   ---------------------------------------- 0.0/60.2 MB ? eta -:--:--
   -- ------------------------------------- 4.5/60.2 MB 22.4 MB/s eta 0:00:03
   ------ --------------------------------- 10.5/60.2 MB 26.2 MB/s eta 0:00:02
   ---------- ----------------------------- 16.5/60.2 MB 27.4 MB/s eta 0:00:02
   -------------- ------------------------- 21.8/60.2 MB 26.5 MB/s eta 0:00:02
   ----------------- ---------------------- 27.0/60.2 MB 25.9 MB/s eta 0:00:02
   --------------------- ------------------ 32.2/60.2 MB 25.6 MB/s eta 0:00:02
   ------------------------ --------------- 37.5/60.2 MB 25.6 MB/s eta 0:00:01
   ---------------------------- ----------- 43.0/60.2 MB 25.6 MB/s eta 0:00:01
   -------------------------------- ------- 49.3/60.2 MB 25.9 MB/s eta 0:00:01
   ------------------------------------ --- 55.3/60.2 MB 26.3 MB/s eta 0:00:01
   ---------------------------------------  60.0/60.2 MB 26.6 MB/s eta 0:00:01
   ---------------------------------------- 60.2/60.2 MB 24.7 

  You can safely remove it manually.


### 📚 Importing Essential Libraries

Now let's import all the required libraries for our RAG implementation:

- **🔢 NumPy**: For numerical operations and array handling
- **📖 Wikipedia**: For fetching knowledge base content
- **🤗 Transformers**: For tokenization and QA models  
- **📝 Sentence Transformers**: For creating semantic embeddings
- **⚡ FAISS**: For efficient vector similarity search


In [3]:
import numpy as np
import wikipedia
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer
import faiss

## 📖 Retrieving Knowledge:

Fetch relevant Wikipedia articles based on a given topic:

In [3]:
def get_wikipedia_content(topic):
    try: 
        page = wikipedia.page(topic)
        return page.content
    except wikipedia.exceptions.PageError:
        return None
    except wikipedia.exceptions.DisambiguationError as e:
        # Handling cases where the topic is ambiguous:
        print(f"Ambiguous topic, Get more specific. Possible options: {e.options}")
        return None

# User Input:
topic = input("Enter a topic to learn about: ")
document = get_wikipedia_content(topic)

if not document:
    print("Could not find information on that topic.")
    exit()

## ✂️ Text Chunking:

Since Wikipedia articles can be long, we will split the text into smaller overlapping chunks for better retrieval:

- 📏 **Smart Segmentation**: Breaking text into manageable pieces (256 tokens each)
- 🔗 **Overlapping Chunks**: Maintaining context between segments (20 token overlap)
- ⚡ **Optimized Processing**: Avoiding token limit warnings and errors
- 🎯 **Better Retrieval**: Smaller chunks = more precise similarity matching

**Process:** Each chunk represents a focused piece of information that can be independently searched and retrieved for answering questions.

In [4]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

def split_text(text, chunk_size = 256, chunk_overlap = 20):
    tokens = tokenizer.tokenize(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunks.append(tokenizer.convert_tokens_to_string(tokens[start:end]))
        if end == len(tokens):
            break
        start = end - chunk_overlap
    return chunks

chunks = split_text(document)
print(f"Total chunks: {len(chunks)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (6573 > 512). Running this sequence through the model will result in indexing errors


Total chunks: 28


## 🧮 Vector Embeddings & Search Index:

### 🔄 Converting Text to Mathematical Vectors

Transforms human-readable text into numerical representations that machines can compare:

**🎯 The Process:**
- 📝 **Sentence Transformers**: Convert each text chunk into 768-dimensional vectors
- 🧠 **Semantic Encoding**: Capture meaning, not just keywords
- ⚡ **FAISS Indexing**: Store vectors for lightning-fast similarity search
- 📊 **L2 Distance**: Measure semantic similarity mathematically

To efficiently search for relevant chunks, we will use Sentence Transformers to convert text into embeddings and store them in a FAISS index:

In [5]:
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = embedding_model.encode(chunks)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

## 🔍 Smart Query Processing & Retrieval

### ❓ From Question to Context

In the retrieval phase When a user asks a question, it will:


**The Retrieval Pipeline:**

1️⃣ **🎯 Query Embedding**: Convert user question to same vector space as knowledge chunks

2️⃣ **⚡ Similarity Search**: FAISS finds the k most semantically similar chunks  

3️⃣ **📋 Context Assembly**: Gather the most relevant information pieces

4️⃣ **🎛️ Customizable Retrieval**: Adjust k (number of chunks) based on needs


In [6]:
query = input("Ask a question about the topic: ")
query_embedding = embedding_model.encode([query])
query_embedding_array = np.array(query_embedding)

k = 3   # Number of chunks to retrieve
distances, indices = index.search(query_embedding_array, k)
retrieved_chunks = [chunks[i] for i in indices[0]]
print("Retrieved chunks: ")
for chunk in retrieved_chunks:
    print("- " + chunk)

Retrieved chunks: 
- ( portugal ), reims ( france ) and voros lobogo ( hungary ). the first european cup match took place on 4 september 1955, and ended in a 3 – 3 draw between sporting cp and partizan. the first goal in european cup history was scored by joao baptista martins of sporting cp. the inaugural final took place at the parc des princes between stade de reims and real madrid on 13 june 1956. the spanish squad came back from behind to win 4 – 3 thanks to goals from alfredo di stefano and marquitos, as well as two goals from hector rial. real madrid successfully defended the trophy next season in their home stadium, the santiago bernabeu, against fiorentina. after a scoreless first half, real madrid scored twice in six minutes to defeat the italians. in 1958, milan failed to capitalise after going ahead on the scoreline twice, only for real madrid to equalise. the final, held in heysel stadium, went to extra time where francisco gento scored the game - winning goal to allow rea

## 🤖 Intelligent Answer Generation

### 🧠 From Context to Precise Answers

The final step where retrieved knowledge becomes intelligent responses:

**🎯 Generation Process:**
- 🤖 **RoBERTa QA Model**: Pre-trained on SQuAD 2.0 dataset for optimal performance
- 📝 **Extractive Answering**: Finds exact answer spans within retrieved context
- 🎛️ **Context Fusion**: Combines multiple relevant chunks into unified context
- 📊 **Confidence Scoring**: Provides reliability metrics for each answer


**🔗 Complete RAG Flow**: `Wikipedia → Chunks → Embeddings → Retrieval → Generation → Answer + Confidence`

In [ ]:
# Implement on Windows:
'''
qa_model_name = "deepset/roberta-base-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)

context = " ".join(retrieved_chunks)
answer = qa_pipeline(question = query, context = context)
print(f"Answer: {answer['answer']}")
'''

Device set to use cpu


Answer: 15


In [ ]:
# For Mac (as pipeline library is not available):
import torch

# Load QA model and tokenizer
qa_model_name = "deepset/roberta-base-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

# Create context
context = " ".join(retrieved_chunks)

# Answer the question directly (without pipeline)
inputs = qa_tokenizer(query, context, return_tensors="pt", truncation=True, max_length=512)
with torch.no_grad():
    outputs = qa_model(**inputs)

answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1
answer_text = qa_tokenizer.convert_tokens_to_string(
    qa_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
)

### 📊 Answer Quality Assessment

Monitor the reliability and confidence of generated answers:


In [ ]:
# Calculate confidence score
start_scores = torch.nn.functional.softmax(outputs.start_logits, dim=1)
end_scores = torch.nn.functional.softmax(outputs.end_logits, dim=1)
confidence = (start_scores[0][answer_start] * end_scores[0][answer_end-1]).item()

print(f"Answer: {answer_text}")
print(f"Confidence: {confidence:.4f}")

# Display confidence score
#print(f"Confidence Score: {answer['score']:.4f} ({answer['score']*100:.1f}%)")

Confidence Score: 0.3278 (32.8%)


## 📊 Simple Visualizations

Let's create some basic visualizations to understand our RAG pipeline better:


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
import re

# Set style for better-looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


C:\Users\sandh\AppData\Local\Temp\ipykernel_18820\1089942319.py:26: UserWarning: Glyph 128260 (\N{ANTICLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCLE ARROWS}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\sandh\anaconda3\Lib\site-packages\IPython\core\pylabtools.py:170: UserWarning: Glyph 128260 (\N{ANTICLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCLE ARROWS}) missing from font(s) Arial.
  fig.canvas.print_figure(bytes_io, **kw)


📊 Pipeline Summary:
Input: 1 Wikipedia article
Chunks created: 28
Embeddings generated: 28
Relevant chunks retrieved: 3
Final answer generated: 1


In [ ]:
# Simple Pipeline Flow Visualization
pipeline_steps = ['Wikipedia\nFetch', 'Text\nChunking', 'Vector\nEmbeddings', 'FAISS\nSearch', 'QA Model\nGeneration']
step_values = [1, len(chunks), len(embeddings), len(retrieved_chunks), 1]
step_colors = ['lightblue', 'lightgreen', 'orange', 'gold', 'lightcoral']

plt.figure(figsize=(12, 6))

# Flow diagram
for i, (step, value, color) in enumerate(zip(pipeline_steps, step_values, step_colors)):
    # Draw boxes
    rect = plt.Rectangle((i*2, 1), 1.5, 1, facecolor=color, edgecolor='black', linewidth=2)
    plt.gca().add_patch(rect)
    
    # Add labels
    plt.text(i*2 + 0.75, 1.5, step, ha='center', va='center', fontweight='bold', fontsize=10)
    plt.text(i*2 + 0.75, 0.5, f'{value}', ha='center', va='center', fontsize=12, fontweight='bold')
    
    # Add arrows
    if i < len(pipeline_steps) - 1:
        plt.arrow(i*2 + 1.6, 1.5, 0.3, 0, head_width=0.1, head_length=0.1, fc='black', ec='black')

plt.xlim(-0.5, len(pipeline_steps)*2)
plt.ylim(0, 3)
plt.title('🔄 RAG Pipeline Flow', fontsize=16, fontweight='bold', pad=20)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"📊 Pipeline Summary:")
print(f"Input: 1 Wikipedia article")
print(f"Chunks created: {len(chunks)}")
print(f"Embeddings generated: {len(embeddings)}")
print(f"Relevant chunks retrieved: {len(retrieved_chunks)}")
print(f"Final answer generated: 1")
